# Inference

In [1]:
# imports
import ast
import os
import pathlib

import neptune
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection.transform import GeneralizedRCNNTransform

from pytorch_faster_rcnn.datasets import ObjectDetectionDatasetSingle, ObjectDetectionDataSet
from pytorch_faster_rcnn.faster_RCNN import get_faster_rcnn_resnet
from pytorch_faster_rcnn.transformations import ComposeDouble
from pytorch_faster_rcnn.transformations import ComposeSingle
from pytorch_faster_rcnn.transformations import FunctionWrapperDouble
from pytorch_faster_rcnn.transformations import FunctionWrapperSingle
from pytorch_faster_rcnn.transformations import apply_nms, apply_score_threshold
from pytorch_faster_rcnn.transformations import normalize_01
from pytorch_faster_rcnn.utils import get_filenames_of_path, collate_single, save_json
from pytorch_faster_rcnn.visual import DatasetViewer
from pytorch_faster_rcnn.visual import DatasetViewerSingle
from pytorch_faster_rcnn.backbone_resnet import ResNetBackbones

In [2]:
# root directory
root = pathlib.Path.cwd()

In [3]:
# parameters
params = {'EXPERIMENT': 'IN-58',  # experiment name, 
          'OWNER': 'bdwumah',  # e.g. johndoe55
          'INPUT_DIR': '/home/bdwumah74/Indiv_Proj/tests/images',  # files to predict
          'PREDICTIONS_PATH': 'predictions',  # where to save the predictions
          'MODEL_DIR': '/home/bdwumah74/Indiv_Proj/neptune/Untitled/IN-33/checkpoints/epoch=14-step=7500.ckpt',  # load model from checkpoint
          'DOWNLOAD': False,  # whether to download from neptune
          'DOWNLOAD_PATH': 'model',  # where to save the model if DOWNLOAD is True
          'PROJECT': 'Individual-Project-1',  # Project name
          }

In [4]:
# input files
inputs = get_filenames_of_path(pathlib.Path(params['INPUT_DIR']))
inputs.sort()

In [5]:
# transformations
transforms = ComposeSingle([
    FunctionWrapperSingle(np.moveaxis, source=-1, destination=0),
    FunctionWrapperSingle(normalize_01)
])

In [6]:
# create dataset
dataset = ObjectDetectionDatasetSingle(inputs=inputs,
                                       transform=transforms,
                                       use_cache=False,
                                       )

In [7]:
# create dataloader
dataloader_prediction = DataLoader(dataset=dataset,
                                   batch_size=1,
                                   shuffle=False,
                                   num_workers=0,
                                   collate_fn=collate_single)

In [8]:
api_key='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4MjEwNWM2NC1jNmM5LTQyYTctOGRlNy1hN2M0YTk2MmY0N2QifQ=='

In [9]:
project_name = f'{params["OWNER"]}/{params["PROJECT"]}'
project_name

'bdwumah/Individual-Project-1'

In [10]:
api_key

'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4MjEwNWM2NC1jNmM5LTQyYTctOGRlNy1hN2M0YTk2MmY0N2QifQ=='

In [11]:
# import experiment from neptune
project_name = f'{params["OWNER"]}/{params["PROJECT"]}'
project = neptune.init(project_qualified_name=project_name, api_token=api_key)  # get project
experiment_id = params['EXPERIMENT']  # experiment id
experiment = project.get_experiments(id=experiment_id)[0]
#properties = experiment.get_properties()
#parameters = experiment.get_parameters()

In [12]:
# rcnn transform
transform = GeneralizedRCNNTransform(min_size=1024,
                                     max_size=1024,
                                     image_mean=ast.literal_eval('[0.485, 0.456, 0.406]'),
                                     image_std=ast.literal_eval('[0.229, 0.224, 0.225]'))

In [13]:
# view dataset
datasetviewer = DatasetViewerSingle(dataset, rccn_transform=None)
datasetviewer.napari()

libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [14]:
# download model from neptune or load from checkpoint
if params['DOWNLOAD']:
    download_path = pathlib.Path(os.getcwd()) / params['DOWNLOAD_PATH']
    download_path.mkdir(parents=True, exist_ok=True)
    model_name = 'epoch=14-step=7500.ckpt'  # that's how I called the best model
    # model_name = properties['checkpoint_name']  # logged when called log_model_neptune()
    if not (download_path / model_name).is_file():
        experiment.download_artifact(path=model_name, destination_dir=download_path)  # download model

    model_state_dict = torch.load(download_path / model_name, map_location=torch.device('cpu'))
else:
    checkpoint = torch.load(params['MODEL_DIR'], map_location=torch.device('cpu'))
    model_state_dict = checkpoint['hyper_parameters']['model'].state_dict()

In [15]:
# model init
model = get_faster_rcnn_resnet(num_classes= 3,
                               backbone_name= ResNetBackbones.RESNET34,  # reverse look-up enum
                               anchor_size=ast.literal_eval('((32, 64, 128, 256, 512),)'),
                               aspect_ratios=ast.literal_eval('((0.5, 1.0, 2.0),)'),
                               fpn=ast.literal_eval('False'),
                               min_size=1024,
                               max_size=1024
                               )

In [16]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(1024,), max_size=1024, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
      

In [17]:
# load weights
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [18]:
# inference (cpu)
model.eval()
for sample in dataloader_prediction:
    x, x_name = sample
    with torch.no_grad():
        pred = model(x)
        pred = {key: value.numpy() for key, value in pred[0].items()}
        name = pathlib.Path(x_name[0])
        save_dir = pathlib.Path(os.getcwd()) / params['PREDICTIONS_PATH']
        save_dir.mkdir(parents=True, exist_ok=True)
        pred_list = {key: value.tolist() for key, value in pred.items()}  # numpy arrays are not serializable -> .tolist()
        save_json(pred_list, path=save_dir / name.with_suffix('.json'))

In [19]:
# get prediction files
predictions = get_filenames_of_path(pathlib.Path(os.getcwd()) / params['PREDICTIONS_PATH'])
predictions.sort()

In [20]:
# create prediction dataset
iou_threshold = 0.25
score_threshold = 0.6

transforms_prediction = ComposeDouble([
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01),
    FunctionWrapperDouble(apply_nms, input=False, target=True, iou_threshold=iou_threshold),
    FunctionWrapperDouble(apply_score_threshold, input=False, target=True, score_threshold=score_threshold)
])

dataset_prediction = ObjectDetectionDataSet(inputs=inputs,
                                            targets=predictions,
                                            transform=transforms_prediction,
                                            use_cache=False)

In [21]:
# mapping
color_mapping = {
    1: 'red',
}

In [22]:
# visualize predictions
datasetviewer_prediction = DatasetViewer(dataset_prediction, color_mapping)
datasetviewer_prediction.napari()
# add text properties gui
datasetviewer_prediction.gui_text_properties(datasetviewer_prediction.shape_layer)

## Experiment with Non-maximum suppression (nms) and score-thresholding

In [ ]:
# experiment with nms and score-thresholding
transforms_prediction = ComposeDouble([
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

dataset_prediction = ObjectDetectionDataSet(inputs=inputs,
                                            targets=predictions,
                                            transform=transforms_prediction,
                                            use_cache=False)

color_mapping = {
    1: 'red',
}

datasetviewer_prediction = DatasetViewer(dataset_prediction, color_mapping)
datasetviewer_prediction.napari()

In [ ]:
# add score slider
# DOES CURRENTLY NOT WORK
# datasetviewer_prediction.gui_score_slider(datasetviewer_prediction.shape_layer)

In [ ]:
# add nms slider
# DOES CURRENTLY NOT WORK
# datasetviewer_prediction.gui_nms_slider(datasetviewer_prediction.shape_layer)